# Multilingual WSD

## Prepare workingspace

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%shell

unrar e -o+ 'code.rar' code/ 

wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
apt-get update
apt-get install cuda=9.0.176-1

pip install --upgrade tensorflow==1.12*
pip install --upgrade tensorflow-gpu==1.12.*
pip uninstall -y keras
pip install keras_self_attention

rm -rf sample_data

mkdir resources data
mkdir data/evaluation data/training
mkdir checkpoint checkpoint_multitask

cp 'drive/My Drive/babelnet2lexnames.tsv' resources/
cp 'drive/My Drive/babelnet2wndomains.tsv' resources/
cp 'drive/My Drive/babelnet2wordnet.tsv' resources/

cp 'drive/My Drive/onesec.tar.gz' 'onesec.tar.gz'
tar -xzf 'onesec.tar.gz'  -C 'data/training/'
rm -rf "onesec.tar.gz"

cp 'drive/My Drive/sensembert_supervised.txt' 'resources/sensembert_supervised.txt'
cp 'drive/My Drive/lemma2synsets4.0.xx.wn.ALL.txt' 'data/training/lemma2synsets4.0.xx.wn.ALL.txt'

cp 'drive/My Drive/WSD_Training_Corpora.zip' WSD_Training_Corpora.zip
unzip -q 'WSD_Training_Corpora.zip' -d data/training/
rm -rf "WSD_Training_Corpora.zip"

cp 'drive/My Drive/WSD_Unified_Evaluation_Datasets.zip' WSD_Unified_Evaluation_Datasets.zip
unzip -q "WSD_Unified_Evaluation_Datasets.zip" -d data/evaluation/
rm -rf "WSD_Unified_Evaluation_Datasets.zip"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from code.rar

Extracting  code/hierarchical_model.py                                     3%  OK 
Extracting  code/keras_attention.py                                        4%  OK 
Extracting  code/keras_bert.py                                             6%  OK 
Extracting  code/keras_elmo.py                                             8%  OK 
Extracting  code/main.py                                                   9%  OK 
Extracting  code/model.py                                                 13%  OK 
Extracting  code/models_embeddings.py                                     18%  OK 
Extracting  code/models_main.py                                           20%  OK 
Extracting  code/model_bert.py                                            29%  OK 
Extracting  code/model_utils.py                                          

In [0]:
import nltk


nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('omw', quiet=True)

True

## Fix config params (if needed)

In [37]:
import yaml
import os


config_file_path = os.path.join(os.getcwd(), "config.yaml")
config_file = open(config_file_path)
config_params = yaml.load(config_file)
config_params['use_elmo']=False
config_params['load_embeddings']=True
config_params['epochs']=10
config_params['batch_size']=8
# config_params['hidden_size']=16
display(config_params)

{'USE_CPU': False,
 'batch_size': 8,
 'embedding_size': 400,
 'epochs': 10,
 'hidden_size': 128,
 'load_embeddings': True,
 'use_bert': False,
 'use_elmo': False}

## Run code

In [40]:
!python code/models_embeddings.py

6249/6250 [============================>.] - ETA: 0s - loss: 2.5225 - WSD_output_loss: 1.3178 - POS_output_loss: 0.6899 - LEX_output_loss: 0.5149 - WSD_output_acc: 0.7872 - POS_output_acc: 0.7578 - LEX_output_acc: 0.8907
Epoch 00001: loss improved from inf to 2.52233, saving model to checkpoint_multitask/SensEmbed_BiLSTM_ATT_MultiTask.hdf5
6250/6250 [==============================] - 3480s 557ms/step - loss: 2.5223 - WSD_output_loss: 1.3177 - POS_output_loss: 0.6898 - LEX_output_loss: 0.5148 - WSD_output_acc: 0.7872 - POS_output_acc: 0.7578 - LEX_output_acc: 0.8908
Epoch 2/5
6249/6250 [============================>.] - ETA: 0s - loss: 1.8119 - WSD_output_loss: 1.0373 - POS_output_loss: 0.3565 - LEX_output_loss: 0.4181 - WSD_output_acc: 0.8443 - POS_output_acc: 0.8837 - LEX_output_acc: 0.9008
Epoch 00002: loss improved from 2.52233 to 1.81173, saving model to checkpoint_multitask/SensEmbed_BiLSTM_ATT_MultiTask.hdf5
6250/6250 [==============================] - 3477s 556ms/step - loss: 1.

In [0]:
!zip -r 'checkpoint_multitask.zip' checkpoint_multitask
!zip -r 'logs.zip' logs
!zip -r 'resources.zip' resources


from google.colab import files
files.download("checkpoint_multitask.zip")
files.download("resources.zip")
files.download("logs.zip")
